# Make City Data
Aggregate data of cities (metropolitan regions, given by a collection of counties) from the county data stored as per `make_county_data.ipynb`.

**File structure**:
- `~/city_wise/`
    - `<city_name>/`
        - `shapefile/`
        - `places.pickle`
        - `census.pickle`
        - `patterns_<start date>_<end date>.pickle`
        - `homes_<start date>_<end date>.pickle`
        - `social_dist_<start date>_<end date>.pickle`
        - `social_od_<start date>_<end date>.pickle`

## Setup

In [8]:
from covid_commons import *
import os
import glob
import json
import geopandas as gp
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from time import time
from importlib import reload

In [9]:
import covid_commons as g
from covid_commons import peek
g = reload(g)

---
## Create cities

In [2]:
class City():
    def __init__(self, key, dict_):
        self.key = key  # key in the cities dictionary
        self.name = dict_['name']
        self.pretty_name = self.name.replace('_', ' ').title()
        self.dir = IO['city_root'] + '/' + self.name
        self.counties = dict_['counties']
        self.events = dict_['events']
        
    def __repr__(self):
        return f'<City:{self.name}>'

In [3]:
with open(IO['city_info'], 'r') as f:
    cities_dict = json.load(f)
    C = {k: City(k, v) for k, v in cities_dict.items()}

### Illinois

In [41]:
il = g.City('il', {
    'name': 'Illinois',
    'events': {},
    'counties': {x[-3:]: [17, int(x[-3:])] for x in
                 glob.glob(g.DATA_DIR+'/county_wise/17/*')}
})

In [57]:
C = {'il': il}

## Aggregate the data

### General functions

#### Load city data

In [17]:
def load_city_data(city, variable, ftype='pickle', dates=None, cbg_var=False):
    """
    Load the data of a given variable in the given city & period.
    @param city: <City>
    @param dates: <pd.DateTimeIndex> dates for which data is to be retrieved
    @param variable: <str> measure of interest
    @param cbg_var: <bool>
    @param ftype: <str> extension of data file (one of 'pickle' or 'csv')
    @return data: <pd.df>
    """
    data = pd.DataFrame()
    for cnty_name, (state, cnty) in tqdm(city.counties.items(), desc=city.name):
        dir_ = f'{IO["cnty_root"]}/{state:02}/{cnty:03}'
        # when the content is static
        if dates is None:
            file = f'{dir_}/{variable}.{ftype}'
            if os.path.exists(file):
                if ftype == 'pickle':
                    data = data.append(pd.read_pickle(file), ignore_index=True)
                elif ftype == 'csv':
                    df = pd.read_csv(file)
                    df.insert(0, 'state', state)
                    df.insert(1, 'cnty', cnty)
                    data = data.append(df)
        # when the content is dynamic (daily/weekly)
        else:
            for date in dates:
                date_str = date.strftime('%Y-%m-%d')
                file = f'{dir_}/{variable}/{variable}_{date_str}.pickle'
                if os.path.exists(file):
                    df = pd.read_pickle(file)
                    df['date'] = int(date.strftime('%y%m%d'))
                    if cbg_var == False:
                        df['state'] = state
                        df['cnty'] = cnty
                        df = df.astype({'state': np.int8, 'cnty': np.int16})
                    df = df.astype({'date': np.int32})
                    data = data.append(df, ignore_index=True)
    return data

#### Save city data

In [18]:
def save_city_data(data, city, fname, ftype='pickle', dates=None):
    """
    Write the combined data of city's counties to disk
    """
    if not os.path.exists(city.dir):
        os.makedirs(city.dir)
    if dates is None:
        file = f'{city.dir}/{fname}.{ftype}'
    else:
        file = f'{city.dir}/{fname}_{dateRange2str(dates)}.{ftype}'
    if ftype == 'pickle':
        data.to_pickle(file)
    elif ftype == 'csv':
        data.to_csv(file, index=False)

## Raw tabular datasets

### $R_t$ (health)

In [59]:
for c in C.values():
    c.rt = load_city_data(c, 'rt', 'csv')
    save_city_data(c.rt, c, 'rt', 'csv')

### Places

In [60]:
for c in C.values():
    c.pois = load_city_data(c, 'places')
    save_city_data(c.pois, c, 'places')

### Census

In [61]:
for c in C.values():
    c.acs = load_city_data(c, 'census')
    save_city_data(c.acs, c, 'census')

### Weekly patterns

In [62]:
%%time
for c in C.values():
    save_city_data(load_city_data(c, 'patterns', dates=g.WEEKS),
                   c, 'patterns', dates=g.WEEKS)


CPU times: user 36min 40s, sys: 3min 31s, total: 40min 11s
Wall time: 42min 20s


### Weekly pattern OD
Origin-destination table of no. of visitors between home CBG & POI

In [63]:
%%time
for c in C.values():
    save_city_data(load_city_data(c, 'homes', dates=g.WEEKS),
                   c, 'patterns_od', dates=g.WEEKS)


CPU times: user 8min 20s, sys: 5min 49s, total: 14min 10s
Wall time: 14min 43s


### Social distancing

In [64]:
%%time
for c in C.values():
    save_city_data(load_city_data(c, 'social_dist', dates=g.DATES),
                   c, 'social_dist', dates=g.DATES)


CPU times: user 37min 12s, sys: 11min 31s, total: 48min 44s
Wall time: 52min 32s


### Social distancing OD
Origin-destination table of no. of devices moving from home CBG to non-home CBG

In [67]:
%%time
for c in C.values():
    save_city_data(load_city_data(c, 'social_od', dates=g.DATES),
                   c, 'social_od', dates=g.DATES)


CPU times: user 3h 24min 26s, sys: 3h 3min 58s, total: 6h 28min 25s
Wall time: 6h 54min 43s


### CBG Shapefile

In [54]:
def get_cbg_shapefile(city, write=True):
    """
    Extract the shapefile of the given city from its state shapefile
    containing the CBGs & save it to the city's folder.
    """
    # get the shapefile by filtering the counties
    shp = []
    for state_fips, cnty_fips in city.counties.values():
        shp_file = g.IO['state_shp'].format(state_fips)
        df = gp.read_file(shp_file)
        shp.append(df[df['COUNTYFP'] == f'{cnty_fips:03}'])
    shp = pd.concat(shp, axis=0, ignore_index=True)
    
    # save to disk
    if write:
        dir_ = f'{city.dir}/shapefile'
        if not os.path.exists(dir_):
            os.makedirs(dir_)
        file = f'{dir_}/{city.name}_CBG.shp'
        shp.to_file(file)
    
    return shp

In [ ]:
%%time
pbar = tqdm(C.values())
for c in pbar:
    pbar.set_description(c.name)
    c.shp = get_cbg_shapefile(c, write=True)

#### Visualize shapefile

In [20]:
def plot_map(city, col, colname=None, cmap='Reds',
             min_=None, max_=None, figsize=(10, 10)):
    """
    Plot the thematic map of the CBGs of a city by a continuous variable.
    @param city: <City> city of interest
    @param col: <str> census variable/column of interest (continuous)
    @param colname: <str> pretty name of the census variable
    @param min_, max_: <float> lower and upper limits on the colorbar
    """
    # join the shapefile with the census field by CBG
    gdf = (city.shp.astype({'GEOID': np.int64})
           .merge(city.acs.set_index('cbg')[col].dropna(),
                  left_on='GEOID', right_index=True))
    
    # cut off extreme values to prevent colormap from exploding
    extend = 'neither'
    if max_ is not None:
        gdf.loc[gdf[col] > max_, col] = max_
        extend = 'max'
    if min_ is not None:
        gdf.loc[gdf[col] < min_, col] = min_
        extend = 'min'
    if max_ is not None and min_ is not None:
        extend = 'both'
        
    fig, ax = plt.subplots(figsize=figsize)
    ax.axis('off')
    
    # create colorbar legend
    vmin, vmax = gdf[col].min(), gdf[col].max()
    sm = plt.cm.ScalarMappable(
        cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2%', pad=0.05)
    plt.colorbar(sm, cax=cax, extend=extend)
    
    # main plot
    gdf.plot(ax=ax, column=col, cmap=cmap, legend=False)
    colname = colname if colname is not None else col
    ax.set_title(f'CBGs by {colname} (2017)')

In [ ]:
%time plot_map(C['nyc'], 'med_hh_income', cmap='Blues')

### County shapefile

In [52]:
def get_cnty_shapefile(city, write=False):
    """
    Extract the shapefile of the counties of a city & save if required.
    """
    # specify how columns are to be aggregated
    agg = {'STATEFP': min, 'COUNTYFP': min, 'ALAND': sum, 'AWATER': sum}
    
    # get the shapefile by filtering the counties
    records = []
    geometries = []
    for state_fips, cnty_fips in city.counties.values():
        shp_file = g.IO['state_shp'].format(state_fips)
        state_df = gp.read_file(shp_file)
        # get the county level dataframe
        df = state_df.query(f'COUNTYFP == "{cnty_fips:03}"')
        # filter out the all water CBGs
        df = df.query('ALAND > 0')
        # compute its unary union (time consuming step)
        union = df['geometry'].unary_union
        geometries.append(union)
        # aggregate other attributes
        records.append(df.agg(agg))
    # create the pandas and geopandas dataframes
    records = pd.DataFrame(records)
    shp = gp.GeoDataFrame(gp.GeoSeries(geometries).rename('geometry'))
    for col in agg.keys():
        shp[col] = records[col]
    
    # save to disk
    if write:
        dir_ = f'{city.dir}/shapefile'
        if not os.path.exists(dir_):
            os.makedirs(dir_)
        file = f'{dir_}/{city.name}_cnty.shp'
        shp.to_file(file)
    
    return shp

In [ ]:
%%time
pbar = tqdm(C.values())
for c in pbar:
    pbar.set_description(c.name)
    c.shp = get_cnty_shapefile(c, write=True)

---
## Exposure mobility metrics

### Prepare visits

In [22]:
def get_pat_poi(city):
    """
    Load the patterns data of a city and isolate its hourly visits matrix
    """
    # get the patterns data
    if not hasattr(city, 'pat') or not 'visits_hourly' in city.pat.columns:
        pat = pd.read_pickle(f'{city.dir}/patterns_{dateRange2str(WEEKS)}.pickle')
        # join with POI table to get floor area
        pois = (city.pois[['poi_id', 'naics', 'includes_parking_lot', 'area_sqft']]
                .rename(columns={'area_sqft': 'area'}))
        city.pat = pat.merge(pois, on='poi_id')
        
    # get the hourly visits matrix
    if not hasattr(city, 'poi_visits_hourly'):
        # pop the column to not occupy more space
        vis = city.pat.pop('visits_hourly')
        # convert to matrix
        vis = np.reshape(np.stack(vis), (vis.shape[0]*7, 24))
        # convert week index to date & format it
        days = (pd.to_datetime(np.repeat(city.pat['date'], 7), format='%y%m%d') +
                pd.to_timedelta(np.tile(np.arange(7), city.pat.shape[0]), unit='d'))
        # prepare the table
        city.visits_hourly = pd.DataFrame(vis, index=days).rename_axis('date')

### Get the exposure metrics

In [25]:
def get_exp_mob(city, midpoints=g.DWELL_BINS['exp_hour'], include_last_bin=True,
                write=False):
    """
    Get the POI-level daily exposure based mobility metrics of a city
    from hourly POI visits, areas, and weekly dwell time distribution matrix.
    @param midpoints: <[]> representative points of required dwell time buckets
    @param include_last_bin: whether count the visits of the last bucket
    (>4 hr) as valid visits >1 h. When false, combine visits of bins 1-4 hr &
    >4 hr into one category with representative point of 1 hr
    """
    # get the patterns data (row: (poi, week))
    pat = city.pat

    # get index of POIs to be filtered out because their area includes parking lot
    idx = pat['includes_parking_lot'] == False

    # get the area of these filtered POIs
    areas = np.repeat(pat[idx]['area'].values, 7)

    # get the hourly visits dataframe & convert it to matrix
    viz_df = city.visits_hourly
    viz_mat = viz_df.values[np.repeat(idx.values, 7), :]

    # convert ones to zeros in this matrix because ones, like zeros, have no
    # contribution in social contact; converting them to zero ensures that
    # those ones do not contribute to the hourly social time becoming -ve
    viz_mat = np.where(viz_mat == 1, 0, viz_mat)

    # get the daily visits
    daily_viz = viz_mat.sum(axis=1)

    # calculate the daily RPS
    with np.errstate(invalid='ignore'):
        rps = np.sqrt(viz_mat.sum(1) * np.sqrt(areas) / daily_viz)

    # get the weekly dwell time distribution matrix for PET
    distr = np.array(pat[idx]['dwell_bins'].tolist()).copy()
    distr = distr / distr.sum(axis=1)[:, None]
    # if required, add visits of 1-4 hr & >4 hr with representative point =1 hr
    if include_last_bin:
        distr[:, 3] = distr[:, 3] + distr[:, 4]
    distr = distr[:, :4]
    d1, d2, d3, d4 = distr.T
    t1, t2, t3, t4 = midpoints

    # compute the weekly alpha coefficient (see the PET formula)
    alpha = (t1 * (d1 ** 2 + 2 * d1 * d2 + 2 * d1 * d3 + 2 * d1 * d4) +
             t2 * (d2 ** 2 + 2 * d2 * d3 + 2 * d3 * d4) +
             t3 * (d3 ** 2 + 2 * d3 * d4) +
             t4 * (d4 ** 2))

    # calculate the weekly beta coefficient (see the PET formula)
    beta = distr @ midpoints

    # apply the main formula of contact time to get the daily averaged social
    # time, keeping in mind that weekly vectors are cast to daily by repeating
    with np.errstate(invalid='ignore'):
        pet = np.repeat(alpha, 7) * ((viz_mat ** 2).sum(1) / (
            daily_viz)) - np.repeat(beta, 7)
    # clip the rare negative values to 0
    pet = np.clip(pet, a_min=0, a_max=None)

    # calculate the POI-daily contact exposure index (CEI)
    # unit: minute-persons per foot
    cei = pet / np.sqrt(areas)

    # create the POI-daily exposure dataframe
    exp_df = (
        pd.DataFrame({
            'date': viz_df.index.rename('date')[np.repeat(idx.values, 7)],
            'cbg': np.repeat(pat[idx]['cbg'].values, 7),
            'poi_id': np.repeat(pat[idx]['poi_id'].values, 7),
            'naics': np.repeat(pat[idx]['naics'].values, 7),
            'area': areas,
            'visits': daily_viz,
            'rps': rps,
            'pet': pet,
            'cei': cei
        })
        .dropna()
        .astype({'visits': np.uint16, 'area': np.int32, 'poi_id': np.int32})
        .set_index(['date', 'cbg', 'poi_id', 'naics'])
    )
    if write:
        if include_last_bin:
            exp_df.to_pickle(city.dir + '/exposure.pickle')
        else:
            exp_df.to_pickle(city.dir + '/exposure4.pickle')
    return exp_df

## Aggregate exposure mobility vars

In [26]:
def agg_exp(city, grp_vars, exp_vars=['rps', 'pet', 'cei'], fname=None):
    """
    Aggregate exposure metrics over a given scope as averages weighted by
    visits in that scope. `grp_vars` defines that scope. Optionally, also
    write the output dataframe to disk by the `fname`.
    """
    df = city.exp.dropna()
    for var in exp_vars:
        df[var+'_x_visits'] = df['visits'] * df[var]
    agg_df = (df.groupby(grp_vars)
              [['visits'] + [x+'_x_visits' for x in exp_vars]].sum())
    for var in exp_vars:
        agg_df[var] = agg_df[var+'_x_visits'] / agg_df['visits']
        del agg_df[var+'_x_visits']

    if fname is not None:
        agg_df.to_pickle(f'{city.dir}/{fname}.pickle')
    
    return agg_df

### Aggregate for all cities
* Over CBGs
* Over industries (NAICS)
* Over both CBGs & NAICS

In [66]:
%%time
for c in tqdm(C.values()):
    t = time()
#     print('-'*20+'\n', c.name)
#     print('getting pat poi')
#     c.pois = pd.read_pickle(c.dir + '/places.pickle')
#     get_pat_poi(c)
#     print('getting exposure metrics')
#     c.exp = get_exp_mob(c, write=True)
#     c.exp = pd.read_pickle(c.dir + '/exposure.pickle')
#     print('getting exposure metrics with 4 bins')
#     c.exp4 = get_exp_mob(c, write=True, include_last_bin=False)
#     print('aggregating by CBG, NAICS & both together')
    agg_exp(c, ['cbg', 'date'], fname='exposure_by_cbg')
    agg_exp(c, ['naics', 'date'], fname='exposure_by_naics')
    agg_exp(c, ['cbg', 'naics', 'date'], fname='exposure_by_cbg_naics')
#     print(f'time elapsed: {(time()-t)/60:.2f} min')


CPU times: user 7.78 s, sys: 2.76 s, total: 10.5 s
Wall time: 12.9 s


---

### RPS
Regularized physical spacing (RPS) at the POI level

### PET
Pessimistic exposure time (PET) at the POI level

### CEI
CEI: Contact Exposure Index

It is the hourly ratio of PET (minutes-persons) by the POI area (sq. ft.).

### RPS

### PET/CEI

### Concat

### Aggregate by industry

### RPS

### PET/CEI

### Concat

### Aggregate by CBG & industry

### RPS

### PET/CEI

### Concat